## Auth 실습문제

기존 movies_blog 프로젝트를 이어서 진행합니다.

#### 1. User Model 대체하기
1. 기존에 존재하는 User Model 은 커스텀해서 사용할 수 없으므로 CustomUser 을 생성한 후 DJango의 기본 UserModel 로 설정한다. 
2. 이미 DB가 생성되어 있는 경우는 UserModel을 변경할 수 없음으로 DB의 테이블을 전부 삭제한다 (스키마를 삭제한 후 다시 생성하는 것도 방법)
3. 관리자계정을 생성하고, 올바르게 생성되었는 지 관리자 페이지에 들어가서 확인한다. 

1. 추후에 User Model을 커스텀하기 위해서 accounts/models.py에 아래 코드를 추가한다.
    ```python
    from django.db import models
    from django.contrib.auth.models import AbstractUser

    # Create your models here.
    class User(AbstractUser):
        pass
    ```
2. 새로 생성한 UserModel을 Django 기본 USerModel로 설정을 바꿔준다.  
movies_blog/settings.py에 아래 코드를 추가한다. 
    ```python
    AUTH_USER_MODEL = 'accounts.User'
    ```

3. 새로 생성한 UserModel을 관리자 페이지에서 확인할 수 있도록 accounts/admin.py 에 아래 코드를 추가한다.
    ```python
    from django.contrib import admin
    from django.contrib.auth.admin import UserAdmin
    from .models import User

    # Register your models here.
    admin.site.register(User, UserAdmin)
    ```

4. 기존에 생성된 테이블이 있다면 모두 삭제한 후 migrations 을 진행한다.
    ```shell
    $ python manage.py makemigrations
    $ python manage.py migrate
    ```

5. 아래 명령어를 통해 슈퍼유저 계정을 생성하고, accounts_user 테이블에 정상적으로 슈퍼유저 계정이 생성되었는 지 확인한다. 
    ```shell
    $ python manage.py createsuperuser
    ```

6. 서버를 킨 후, 관리자 페이지에 들어가서 Custom한 UserModel 을 확인한다.

#### 2. 로그인 기능 구현하기 
1. 기존에 생성되어 있는 login 페이지를 django 로그인 Form 을 이용하여 구현한다. 
2. 로그인 성공 후 accounts/info/ url에서 로그인한 계정 정보를 확인할 수 있도록 한다. 

<img src="practice_images/image39.jpg" width="30%" height="1%"/>
<img src="practice_images/image40.jpg" width="30%" height="1%"/>

1. 이미 accounts:login url은 작성되어 있으므로, AuthenticationForm 을 이용해서 로그인 화면을 랜더링하는 함수로 accounts/views.py의 login함수를 아래와 같이 수정한다.
    ```python
    from django.shortcuts import render
    from django.contrib.auth.forms import AuthenticationForm

    # accounts/views.py
    def login(request):
        if request.method == "POST":
            pass
        else:
            form = AuthenticationForm()
        context = {
            "form": form
        }
        return render(request, "accounts/login.html", context)
    ```

2. 1번에서 context로 건네준 form을 이용해서 로그인 페이지를 구현한다.  
accounts/login.html을 아래와 같이 수정한다. 
    ```html
    <!-- accounts/login.html-->
    {% extends "base.html" %}

    {% block content %}
        <form action="{% url "accounts:login" %}" method="POST">
            {% csrf_token %}
            {{ form.as_p }}
            <input type="submit" value="로그인하기">
        </form>
    {% endblock content %}
    ```

3. accounts/login.html 에서 "accounts:login" url에 post 요청으로 보내질 경우의 코드를 작성한다  
accounts/views.py의 login 함수를 아래와 같이 수정한다.
    ```python
    from django.contrib.auth.forms import AuthenticationForm
    from django.contrib.auth import login as auth_login

    # accounts/views.py
    def login(request):
        if request.method == "POST":
            form = AuthenticationForm(request, request.POST)
            if form.is_valid():
                auth_login(request, form.get_user())
                return redirect('accounts:index')
        else:
            form = AuthenticationForm()
        context = {
            "form": form
        }
        return render(request, "accounts/login.html", context)
    ```

4. login 함수가 redirect할 accounts:index url을 생성한다.  
accounts/urls.py 의 urlpatterns에 아래 코드를 추가한다.
    ```python
    path('index/', views.index, name='index')
    ```

5. index url이 호출할 index 함수를 생성한다.  
accounts/views.py 에 index 함수를 생성한다. 
    ```python
    def index(request):
        return render(request, "accounts/index.html")
    ```

6. index 함수가 랜더링하는 accounts/index.html을 생성한다. 
    ```html
    <!-- accounts/index.html-->
    {% extends "base.html" %}
    {% block content %}
        <h1>회원정보 페이지</h1>
        <hr>
        <h2>{{ user.username}}</h2>
    {% endblock content %}
    ```

#### 3. 로그아웃 기능 구현하기 
1. 회원정보 페이지에 로그아웃 태그를 추가하여 로그아웃 기능을 구현한다.

2. 로그아웃이 완료되면 로그인 페이지로 리다이렉트한다.  

<img src="practice_images/image41.jpg" width="30%" height="1%"/>

1. 회원정보 페이지에 로그아웃 태그 추가하기  
accounts/index.html 코드 수정
    ```html
    <!-- accounts/index.html-->
    {% extends "base.html" %}
    {% block content %}
    <h1>회원정보 페이지</h1>
    <hr>
    <h2>{{ user.username}}</h2>
    <hr>
    <form action="{% url "accounts:logout" %}" mehtod="POST">
        {% csrf_token %}
        <input type="submit" value="로그아웃하기">
    </form><br>
    {% endblock content %}
    ```

2. index.html 에서 필요한 accounts:logout url 을 accounts/urls.py의 urlpatterns에 아래 코드 추가하기
    ```python
    path('logout/', views.logout, name='logout')
    ```

3. logout url에서 필요한 logout 함수를 accounts/views.py에 추가하기
    ```python
    # accounts/views.py
    from django.contrib.auth import logout as auth_logout

    def logout(request):
        auth_logout(request)
        return redirect("accounts:login")
    ```

#### 4. 회원가입 기능 구현하기 
1. 로그인 페이지에 회원가입 태그를 추가한다. 
2. 아이디, 이메일, 비밀번호를 입력받고, 회원가입을 진행한다.
3. 회원가입이 완료되면 회원정보 페이지로 리다이렉트하며, 회원정보 아이디, 이메일이 출력되도록 한다. 

<img src="practice_images/image45.jpg" width="30%" height="1%"/>
<img src="practice_images/image46.jpg" width="30%" height="1%"/>
<img src="practice_images/image47.jpg" width="30%" height="1%"/>

1. accounts/login.html에 회원가입 태그를 추가한다.
    ```html
    <!-- accounts/login.html-->
    {% extends "base.html" %}

    {% block content %}
        <form action="{% url "accounts:login" %}" method="POST">
            {% csrf_token %}
            {{ form.as_p }}
            <input type="submit" value="로그인하기">
        </form>
        <a href="{% url "accounts:signup" %}">회원가입하기</a>
    {% endblock content %}
    ```

2. login.html 에서 필요한 accounts:signup url을 추가한다.
accounts/urls.py 의 urlpatterns에 아래 코드를 추가한다. 
    ```python
    path('signup/', views.signup, name='signup'),
    ```

3. 회원가입에 사용되는 UserCreationForm 은 기본 UserModel이 기본이므로, Custom User Model 을 기반으로 하는 CustomUserCreationForm을 생성한다.
accounts app에 forms.py 파일을 만들고, 안에 아래 코드를 추가한다.
    ```python
    from django.contrib.auth.forms import UserCreationForm
    from django.contrib.auth import get_user_model

    # accounts/forms.py
    class CustomUserCreationForm(UserCreationForm):
        class Meta(UserCreationForm.Meta):
            model = get_user_model()
    ```

4. signup url 에게 필요한 signup 함수를 추가한다.
accounts/views.py 에 singup 함수를 생성한다. 이 때 3번에서 생성한 CustomCreationForm을 사용한다. 
우선 회원가입 페이지 로드를 위해 GET 메소드 영역만 작성한다.
    ```python
    from .forms import CustomUserCreationForm

    # accounts/views.py
    def signup(request):
        if request.method == "POST":
            pass
        else:
            form = CustomUserCreationForm()
        context = {
            'form': form
        }
        return render(request, 'accounts/signup.html', context)
    ```

5. accounts/views.py 의 signup 함수에 필요한 signup.html 파일을 생성한다.
    ```html
    <!-- accounts/signup.html-->
    {% extends "base.html" %}

    {% block content %}
        <form action="{% url "accounts:signup" %}" method="POST">
            {% csrf_token %}
            {{ form.as_p }}
            <input type="submit" value="회원가입하기">
        </form>
    {% endblock content %}
    ```

6. form 제출했을 때의 경로 accounts:signup url의 post 구문을 작성한다.
이미 accounts:signup은 작성되어 있으니, 해당 url이 호출하는 accounts/views.py의 signup 함수에 post 일 경우의 로직을 작성한다. 
    ```python
    from .forms import CustomUserCreationForm

    # accounts/views.py
    def signup(request):
        if request.method == "POST":
            form = CustomUserCreationForm(request.POST)
            if form.is_valid():
                form.save()
                return redirect('accounts:login')
        else:
            form = CustomUserCreationForm()
        context = {
            'form': form
        }
        return render(request, 'accounts/signup.html', context)
    ```

7. 회원정보에 이메일 데이터를 추가한다.
accounts/index.html 코드를 수정한다.
    ```html
    <!-- accounts/index.html-->
    {% extends "base.html" %}
    {% block content %}
    <h1>회원정보 페이지</h1>
    <hr>
    <h2>{{ user.username}}</h2>
    <h2>{{ user.email}}</h2>
    <hr>
    <form action="{% url "accounts:logout" %}" mehtod="POST">
        {% csrf_token %}
        <input type="submit" value="로그아웃하기">
    </form><br>
    {% endblock content %}
    ```


#### 4. 회원탈퇴 기능 구현하기 
1. 회원정보 페이지에 회원탈퇴 태그를 추가한다.
2. 회원탈퇴를 진행한 후, 로그인 페이지로 리다이렉트한다.

<img src="practice_images/image48.jpg" width="30%" height="1%"/>

1. accounts/index.html 에 회워탈퇴하기 태그를 추가한다.
    ```html
    <!-- accounts/index.html-->
    {% extends "base.html" %}
    {% block content %}
    <h1>회원정보 페이지</h1>
    <hr>
    <h2>{{ user.username}}</h2>
    <h2>{{ user.email}}</h2>
    <hr>
    <form action="{% url "accounts:logout" %}" mehtod="POST">
        {% csrf_token %}
        <input type="submit" value="로그아웃하기">
    </form><br>
    <form action="{% url "accounts:delete" %}" mehtod="POST">
        {% csrf_token %}
        <input type="submit" value="회원탈퇴하기">
    </form><br>
    {% endblock content %}
    ```

2. index.html에서 필요한 accounts:delete url을 accounts/urls.py 의 urlpatterns 에 아래 코드를 추가한다.
    ```python
    path('delete/', views.delete, name='delete'),
    ```

3. delete url 에서 필요한 delete 함수를 accounts/views.py에 delete 함수를 작성한다.
    ```python
    # accounts/views.py
    def delete(request):
        request.user.delete()
        return redirect('accounts:login')
    ```


#### 5. 회원정보 수정 구현하기 
1. 회원정보 페이지에 회원정보 수정 태그를 추가한다.
2. 회원정보 수정 페이지에 들어갈 경우, 기존 데이터가 출력되도록 한다.
3. 회원정보 페이지에서 이메일과 비밀번호만 수정할 수 있도록 한다.
4. 수정이 완료되면 회원정보 페이지로 리다리엑트 한다.

<img src="practice_images/image50.jpg" width="30%" height="1%"/>
<img src="practice_images/image49.jpg" width="30%" height="1%"/>

1. accounts/index.html 에 회원정보 수정 태그를 추가한다.
    ```html
    <!-- accounts/index.html-->
    {% extends "base.html" %}
    {% block content %}
    <h1>회원정보 페이지</h1>
    <hr>
    <h2>{{ user.username}}</h2>
    <h2>{{ user.email}}</h2>
    <hr>
    <form action="{% url "accounts:update" %}" mehtod="POST">
        {% csrf_token %}
        <input type="submit" value="회원정보 수정하기">
    </form><br>
    <form action="{% url "accounts:logout" %}" mehtod="POST">
        {% csrf_token %}
        <input type="submit" value="로그아웃하기">
    </form><br>
    <form action="{% url "accounts:delete" %}" mehtod="POST">
        {% csrf_token %}
        <input type="submit" value="회원탈퇴하기">
    </form><br>
    {% endblock content %}
    ```
2. index.html 에서 필요한 accounts:update url을 accounts/urls.py의 urlpatterns 안에 아래 코드를 추가한다.
    ```python
    path('update/', views.update, name="update")
    ```

3. 유저 정보 업데이트에 사용되는 UserChangeForm은 기본 UserModel을 사용하므로, CustomUser를 모델로하는 CustomUserChangeForm을 생성한다.
이메일만 수정할 수 있도록 fiels를 조정한다.  

    ```python
        from django.contrib.auth.forms import UserCreationForm, UserChangeForm
    from django.contrib.auth import get_user_model

    # accounts/forms.py
    class CustomUserChangeForm(UserChangeForm):
        class Meta(UserChangeForm.Meta):
            model = get_user_model()
            fields = ['email'] 
    ```

4. update url에서 필요한 update 함수를 accounts/views.py에 생성한다.  
이 때, 업데이트 화면 (GET)만 구현하고, CustomUserChangeForm을 사용한다.

    ```python
    # accounts/views.py
    from .forms import CustomUserChangeForm

    def update(request):
        if request.method == "POST":
            pass
        else:
            form = CustomUserChangeForm(instance=request.user)
        context = {
            'form': form
        }
        return render(request, "accounts/update.html", context)
    ```
5. update 함수에서 필요한 update.html을 생성한다. 
    ```html
    <!-- accounts/update.html -->
    {% extends "base.html" %}
    {% block content %}
        <form action="{% url "accounts:update" %}" method="POST">
            {% csrf_token %}
            {{ form.as_p }}
            <input type="submit" value="수정하기">
        </form>
    {% endblock content %}
    ```
6. 업데이트 화면이 잘 나타나는 지 확인한다. 
7. accounts/update url이 필요로 하는 accounts/urls.py 의 update 함수에 POST 영역을 작성한다.
    ```python
    # accounts/views.py
    from .forms import CustomUserChangeForm

    def update(request):
        if request.method == "POST":
            form = CustomUserChangeForm(request.POST, instance=request.user)
            if form.is_valid():
                form.save()
                return redirect('accounts:index')
        else:
            form = CustomUserChangeForm(instance=request.user)
        context = {
            'form': form
        }
        return render(request, "accounts/update.html", context)
    ```


#### 6. 비밀번호 수정 구현하기 
1. 회원정보 수정 시 나타나는 비밀번호 수정 링크를 클릭할 경우, 정상적으로 비밀번호 수정 페이지로 이동할 수 있도록 한다. 
2. 비밀번호를 수정한 뒤에도 세션이 유지되도록 한다.
3. 비밀번호를 수정한 뒤에는 회원정보 페이지로 리다이렉트한다.

<img src="practice_images/image51.jpg" width="30%" height="1%"/>

1. 회원정보 수정 페이지에서 비밀번호 변경 링크를 누르면 아래와 같이 나온다.  
http://localhost:8000/3/password/  
위 경로에서 동작하기 위해서 movies_blog/urls.py에 아래 코드를 작성한다.
    ```python
    from django.contrib import admin
    from django.urls import path, include
    from accounts import views

    urlpatterns = [
        path('admin/', admin.site.urls),
        path('movies/', include('movies.urls')),
        path('accounts/', include('accounts.urls')),
        path('reviews/', include('reviews.urls')),
        path('<int:user_id>/password/', views.change_password, name="change_password"),
    ]
    ```
2. change_password url이 필요로 하는 change_password 함수를 accounts/views.py 에 추가한다. (우선 GET에 해당하는 부분만 작성한다.)
    ```python
    # accounts/views.py
    from django.contrib.auth.forms import PasswordChangeForm

    def change_password(request, user_id):
        if request.method == "POST":
            pass
        else:
            form = PasswordChangeForm(request.user)
        context = {
            'form': form
        }
        return render(request, 'accounts/change_password.html', context)
    ```

3. change_password 함수에서 필요로 하는 change_password.html 생성한다.
    ```html
    <!-- accounts/change_password.html -->
    {% extends "base.html" %}
    {% block content %}
        <form action="{% url "change_password" user_id=user.pk %}" method="POST">
            {% csrf_token %}
            {{ form.as_p }}
            <input type="submit" value="변경하기">
        </form>
    {% endblock content %}
    ```
4. 비밀번호 변경 페이지가 잘 열리는지 확인한다.
5. 변경하기를 눌렀을 때를 작성하기 위해서 accounts/views.py 의 change_password에 POST 부분을 작성한다.
    ```python
    # accounts/views.py
    from django.contrib.auth.forms import PasswordChangeForm
    from django.contrib.auth import update_session_auth_hash

    def change_password(request, user_id):
        if request.method == "POST":
            form = PasswordChangeForm(request.user,request.POST)
            if form.is_valid():
                user = form.save()
                update_session_auth_hash(request, user)
                return redirect('accounts:index')
        else:
            form = PasswordChangeForm(request.user)
        context = {
            'form': form
        }
        return render(request, 'accounts/change_password.html', context)    
    ```

